In [77]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [78]:
#定数を定義
BINS = 10000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合

In [79]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_filename_and_Hz(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    filename_and_Hz=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            filename_and_Hz.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return filename_and_Hz

In [80]:
#ファイル名と周波数を分けて出力する関数
def divide_filename_and_Hz(filename_and_Hz: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in filename_and_Hz:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [81]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [82]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [83]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))    #100000倍して誤差を取る
        index += 1

    return DifferenceAcc

In [84]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [85]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [86]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [87]:
#推定精度を算出する巻数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> float:
    counter = 0
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            print(f"間違ってるやつは{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100  #精度を100分率で返す

In [88]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    print(DifferenceAcc_hist[0])

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [89]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化


    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    print(f"KLダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    print(f"JSダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))}%です")

In [99]:
#KLダイバージェンスとJSダイバージェンスの小さい方を出す関数
def KL_or_JS(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    result =  [[0.0 for j in range(len(filename))] for i in range(len(filename))]

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #KLダイバージェンスとJSダイバージェンスの小さい方を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            result[i][j] = min(resultKLD[i][j], resultJSD[i][j])

    #結果を出力
    df_result = pd.DataFrame(result, index=Hz, columns=Hz)
    display(df_result)
    print(f"KLダイバージェンスとJSダイバージェンスの小さい方を使う方法による推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_result))}%です")

In [90]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram(DifferenceAcc_list)
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [98]:
path = "my_walk_data/"

In [100]:
KL_and_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.225516,0.338564,0.182505,0.082131,0.210642,0.173263,0.089256,0.362728,0.396801,0.143028,0.102852,0.167255,0.180049,1.532515,1.392748,1.240628,2.148453,0.514363,0.232199
100Hz,0.207287,0.000000,0.409151,0.536019,0.133488,0.037274,0.521609,0.080175,0.786783,0.868945,0.346804,0.105952,0.133718,0.308161,2.039807,1.786971,1.413961,2.328279,0.778911,0.318005
100Hz,0.112024,0.342078,0.000000,0.237557,0.115238,0.288170,0.185305,0.201201,0.291912,0.464302,0.217198,0.204278,0.249690,0.296954,1.665031,1.433425,1.423182,2.021235,0.668652,0.394682
100Hz,0.466949,0.742522,1.159773,0.000000,0.417280,0.797052,0.219916,0.383862,0.590104,0.164775,0.231257,0.395448,0.542943,0.202873,0.863445,0.778927,0.760971,1.083899,0.330847,0.232908
100Hz,0.105766,0.178032,0.290943,0.234053,0.000000,0.165483,0.230053,0.080314,0.433013,0.479494,0.162186,0.088181,0.133457,0.174985,1.527285,1.341353,1.174630,1.796806,0.550216,0.225328
100Hz,0.168421,0.022030,0.337640,0.490780,0.101292,0.000000,0.468924,0.063618,0.703549,0.813659,0.306338,0.080218,0.102186,0.280647,1.943583,1.738741,1.380534,2.247759,0.741282,0.291562
50Hz,0.278937,0.612671,0.758585,0.074741,0.301476,0.620450,0.000000,0.299228,0.356728,0.181518,0.146214,0.291060,0.381221,0.177839,0.989190,0.879596,0.908304,1.290865,0.365052,0.250146
50Hz,0.124194,0.117988,0.417522,0.276164,0.083568,0.124451,0.283497,0.000000,0.545922,0.535476,0.182491,0.067342,0.119580,0.157749,1.558488,1.381687,1.159756,1.965241,0.542935,0.202119
50Hz,0.295930,0.775702,0.557632,0.118718,0.357178,0.702452,0.113503,0.433552,0.000000,0.138215,0.145847,0.360195,0.422974,0.275802,1.002269,0.914254,0.999640,1.232799,0.434864,0.329211
50Hz,0.676629,1.078888,1.720173,0.109486,0.690139,1.161650,0.291086,0.603755,0.717257,0.000000,0.282777,0.564151,0.746189,0.272714,0.618153,0.577637,0.625127,0.805204,0.266464,0.277692


間違ってるやつは100Hzと50Hzです
間違ってるやつは100Hzと50Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは10Hzと50Hzです
間違ってるやつは10Hzと100Hzです
間違ってるやつは10Hzと100Hzです
KLダイバージェンスによる推定精度は60.0%です


,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.045086,0.028195,0.044178,0.010060,0.038046,0.034364,0.015093,0.066290,0.095560,0.028037,0.016871,0.025038,0.041263,0.328897,0.306454,0.293078,0.372469,0.137754,0.056731
100Hz,0.045086,0.000000,0.081275,0.128313,0.029834,0.003037,0.119452,0.017410,0.170553,0.198856,0.086137,0.023673,0.029579,0.080986,0.401421,0.377570,0.342550,0.438795,0.202404,0.086584
100Hz,0.028195,0.081275,0.000000,0.071518,0.029344,0.069338,0.054077,0.050602,0.078496,0.133716,0.061537,0.052844,0.063311,0.084742,0.382142,0.357480,0.354052,0.425590,0.190620,0.110042
100Hz,0.044178,0.128313,0.071518,0.000000,0.054268,0.118950,0.013289,0.064338,0.033221,0.022329,0.017959,0.056023,0.060540,0.027173,0.197718,0.187196,0.188477,0.245477,0.070533,0.038376
100Hz,0.010060,0.029834,0.029344,0.054268,0.000000,0.023402,0.045971,0.009765,0.081034,0.111701,0.032255,0.010084,0.017648,0.041079,0.334449,0.310688,0.293556,0.375917,0.145045,0.054821
100Hz,0.038046,0.003037,0.069338,0.118950,0.023402,0.000000,0.108800,0.014789,0.154996,0.188762,0.077904,0.019119,0.025129,0.076356,0.397806,0.373967,0.341493,0.436061,0.197184,0.082355
50Hz,0.034364,0.119452,0.054077,0.013289,0.045971,0.108800,0.000000,0.058467,0.027060,0.035026,0.017527,0.051115,0.056990,0.033804,0.235724,0.220172,0.227818,0.283765,0.092049,0.052024
50Hz,0.015093,0.017410,0.050602,0.064338,0.009765,0.014789,0.058467,0.000000,0.102533,0.121065,0.037680,0.006780,0.012828,0.037399,0.327922,0.306022,0.284388,0.371186,0.142024,0.048922
50Hz,0.066290,0.170553,0.078496,0.033221,0.081034,0.154996,0.027060,0.102533,0.000000,0.040159,0.036497,0.088118,0.097585,0.071348,0.265676,0.248989,0.266975,0.312867,0.123746,0.087710
50Hz,0.095560,0.198856,0.133716,0.022329,0.111701,0.188762,0.035026,0.121065,0.040159,0.000000,0.041611,0.106126,0.108156,0.051662,0.154315,0.151322,0.165511,0.203789,0.062205,0.058811


間違ってるやつは100Hzと50Hzです
間違ってるやつは100Hzと50Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは10Hzと50Hzです
間違ってるやつは10Hzと50Hzです
間違ってるやつは10Hzと50Hzです
JSダイバージェンスによる推定精度は65.0%です


In [101]:
KL_or_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.045086,0.028195,0.044178,0.010060,0.038046,0.034364,0.015093,0.066290,0.095560,0.028037,0.016871,0.025038,0.041263,0.328897,0.306454,0.293078,0.372469,0.137754,0.056731
100Hz,0.045086,0.000000,0.081275,0.128313,0.029834,0.003037,0.119452,0.017410,0.170553,0.198856,0.086137,0.023673,0.029579,0.080986,0.401421,0.377570,0.342550,0.438795,0.202404,0.086584
100Hz,0.028195,0.081275,0.000000,0.071518,0.029344,0.069338,0.054077,0.050602,0.078496,0.133716,0.061537,0.052844,0.063311,0.084742,0.382142,0.357480,0.354052,0.425590,0.190620,0.110042
100Hz,0.044178,0.128313,0.071518,0.000000,0.054268,0.118950,0.013289,0.064338,0.033221,0.022329,0.017959,0.056023,0.060540,0.027173,0.197718,0.187196,0.188477,0.245477,0.070533,0.038376
100Hz,0.010060,0.029834,0.029344,0.054268,0.000000,0.023402,0.045971,0.009765,0.081034,0.111701,0.032255,0.010084,0.017648,0.041079,0.334449,0.310688,0.293556,0.375917,0.145045,0.054821
100Hz,0.038046,0.003037,0.069338,0.118950,0.023402,0.000000,0.108800,0.014789,0.154996,0.188762,0.077904,0.019119,0.025129,0.076356,0.397806,0.373967,0.341493,0.436061,0.197184,0.082355
50Hz,0.034364,0.119452,0.054077,0.013289,0.045971,0.108800,0.000000,0.058467,0.027060,0.035026,0.017527,0.051115,0.056990,0.033804,0.235724,0.220172,0.227818,0.283765,0.092049,0.052024
50Hz,0.015093,0.017410,0.050602,0.064338,0.009765,0.014789,0.058467,0.000000,0.102533,0.121065,0.037680,0.006780,0.012828,0.037399,0.327922,0.306022,0.284388,0.371186,0.142024,0.048922
50Hz,0.066290,0.170553,0.078496,0.033221,0.081034,0.154996,0.027060,0.102533,0.000000,0.040159,0.036497,0.088118,0.097585,0.071348,0.265676,0.248989,0.266975,0.312867,0.123746,0.087710
50Hz,0.095560,0.198856,0.133716,0.022329,0.111701,0.188762,0.035026,0.121065,0.040159,0.000000,0.041611,0.106126,0.108156,0.051662,0.154315,0.151322,0.165511,0.203789,0.062205,0.058811


間違ってるやつは100Hzと50Hzです
間違ってるやつは100Hzと50Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは50Hzと100Hzです
間違ってるやつは10Hzと50Hzです
間違ってるやつは10Hzと50Hzです
間違ってるやつは10Hzと50Hzです
KLダイバージェンスとJSダイバージェンスの小さい方を使う方法による推定精度は65.0%です


In [103]:
random_forest(path)

[0. 0. 0. ... 0. 0. 0.]
正解率 =  0.5
